In [1]:
import numpy as np
import pandas as pd
import joblib
import json
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys

#Savepath
save_path = '/Users/francisco/Library/CloudStorage/OneDrive-Pessoal/Documentos/LabMol/SOFIA/MQ/Paper OdorSight/DATA/Y-rand/'

# Load the model
model_base = joblib.load('/Users/francisco/Library/CloudStorage/OneDrive-Pessoal/Documentos/LabMol/SOFIA/MQ/Paper OdorSight/DATA/Modeling/XGB_best_model.pkl')

df = pd.read_csv('/Users/francisco/Library/CloudStorage/OneDrive-Pessoal/Documentos/LabMol/SOFIA/MQ/Paper OdorSight/DATA/Curation/curated_dataset_final.csv')


df['mol'] = [Chem.MolFromSmiles(x) for x in df['SMILES']]
fps = np.array([MACCSkeys.GenMACCSKeys(x) for x in df['mol']])

x = fps
y = df['Outcome']

x = pd.DataFrame(x)
y = pd.DataFrame(y)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, train_size=0.8, random_state=42, stratify=y)

y_train = np.array(y_train).ravel()
y_test = np.array(y_test).ravel()


In [2]:
from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef, f1_score
from sklearn.base import clone
import os

# Definir o número de rounds de y-randomization
n_rounds = 100

# Inicializar dataframe para armazenar os resultados
resultados = pd.DataFrame(columns=['Round', 'Balanced Accuracy', 'MCC', 'F1 Score'])

# Realizar os rounds de y-randomization
for i in range(n_rounds):
    # Embaralhar as variáveis dependentes no dataset de treino e ajustar o modelo (clonando o pipeline original)
    y_teste_random = np.random.permutation(y_train)

    model_y = clone(model_base)     # clone evita passar parâmetros inválidos para o estimador
    model_y.fit(x_train, y_teste_random)

    y_pred = model_y.predict(x_test)

    # Calcular as métricas de desempenho
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Apender os resultados ao dataframe
    resultados.loc[i] = [i+1, balanced_accuracy, mcc, f1]

# Garantir que o diretório de saída exista e salvar os resultados
os.makedirs(save_path, exist_ok=True)
out_file = os.path.join(save_path, 'y_random_results.xlsx')
resultados.to_excel(out_file, index=False)
print(resultados)

    Round  Balanced Accuracy       MCC  F1 Score
0     1.0           0.498234 -0.003242  0.703314
1     2.0           0.489407 -0.020479  0.708088
2     3.0           0.465454 -0.062802  0.683585
3     4.0           0.454214 -0.081689  0.673332
4     5.0           0.449623 -0.095333  0.682712
..    ...                ...       ...       ...
95   96.0           0.478107 -0.040791  0.695413
96   97.0           0.460040 -0.075110  0.687220
97   98.0           0.491349 -0.016154  0.703010
98   99.0           0.455685 -0.080250  0.677519
99  100.0           0.503943  0.007462  0.712387

[100 rows x 4 columns]
